In [1]:
from transformers import AutoTokenizer,AutoModel
import torch
from peft import get_peft_model, LoraConfig, TaskType
model = AutoModel.from_pretrained(
    "/disc1/models/chatglm-6b", trust_remote_code=True).half().cuda()
tokenizer = AutoTokenizer.from_pretrained("/disc1/models/chatglm-6b", trust_remote_code=True)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=['query_key_value',],
)
model = get_peft_model(model, peft_config)

# 在这里加载lora模型，注意修改chekpoint
peft_path = "/disc1/chatglm_output/adgen-chatglm-6b-lora-5e-4/checkpoint-30/chatglm-lora.pt"
model.load_state_dict(torch.load(peft_path), strict=False)
model.eval()

/home/lingtongtao/miniconda3/envs/chatglm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:09<00:00,  1.19s/it]
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (word_embeddings): Embedding(130528, 4096)
        (layers): ModuleList(
          (0-27): 28 x GLMBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): SelfAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embeddi

In [2]:
text ="为什么冰红茶和柠檬茶的味道一样？"

with torch.autocast("cuda"):
    res, history = model.chat(tokenizer=tokenizer, query=text,max_length=300)
    print(res)

The dtype of attention mask (torch.int64) is not bool


冰红茶和柠檬茶的味道并不完全相同，虽然它们都是带有柠檬味道的饮料，但冰红茶和柠檬茶的制作方法和成分略有不同。

冰红茶是将红茶茶包浸泡在水中，过滤出茶渣后加入冰块，加入牛奶和糖浆制成。而柠檬茶则是将柠檬片加入热水中，加入糖浆和冰块制成。虽然柠檬茶和冰红茶都有柠檬的味道，但柠檬茶的柠檬片会更多，会加入一些柠檬汁，口感更加清爽。

此外，冰红茶和柠檬茶在制作过程中的温度和时间也有所不同，冰红茶会过滤出红茶茶渣，制作过程会更快，而柠檬茶则是先泡制柠檬片，再加入糖浆和冰块，制作时间会更长。

因此，虽然冰红茶和柠檬茶的味道相似，但制作方法和成分有所不同，它们的味道还是有所不同的。
